In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time

def setup_save_directory():
    """
    ユーザーとの対話を通じて、画像の保存先ディレクトリを設定する関数。
    Returns:
        str: 有効な保存先ディレクトリのパス。
    """
    while True:
        choice = input("保存先ディレクトリを選択してください (1: 新規作成, 2: 既存のディレクトリを指定): ")
        if choice == '1':
            # --- 新しいディレクトリを作成する場合 ---
            new_dir_name = input("作成する新しいディレクトリのパスを入力してください: ")
            try:
                os.makedirs(new_dir_name, exist_ok=True)
                print(f"ディレクトリ '{new_dir_name}' を作成しました。")
                return new_dir_name
            except OSError as e:
                print(f"エラー: ディレクトリの作成に失敗しました。 {e}")
                # 作成に失敗した場合はループを継続して再入力を促す
        
        elif choice == '2':
            # --- 既存のディレクトリを指定する場合 ---
            while True:
                existing_dir = input("保存先となる既存のディレクトリのパスを入力してください: ")
                if os.path.isdir(existing_dir):
                    print(f"既存のディレクトリ '{existing_dir}' を使用します。")
                    return existing_dir
                else:
                    print("エラー: 指定されたパスはディレクトリではありません、または存在しません。もう一度入力してください。")
        
        else:
            print("エラー: '1' または '2' を入力してください。")

# --- メインの処理 ---
# 1. 対象のWebページ
TARGET_URL = "https://www.ap-siken.com/apkakomon.php"  # 任意の画像を含むページに変更する

# 2. 保存先ディレクトリをユーザーに選択
SAVE_DIR = setup_save_directory()

# 3. HTML取得
print(f"\n'{TARGET_URL}' からHTMLを取得しています...")
try:
    response = requests.get(TARGET_URL)
    response.raise_for_status()  # ステータスコードが200番台以外の場合は例外を発生させる
except requests.exceptions.RequestException as e:
    print(f"エラー: Webページの取得に失敗しました。 {e}")
    exit() # プログラムを終了

soup = BeautifulSoup(response.content, "html.parser")

# 4. 画像タグをすべて取得
img_tags = soup.find_all("img")
print(f"{len(img_tags)} 個の画像が見つかりました。ダウンロードを開始します。")

# 5. 画像URLごとに処理
saved_count = 0
for i, img in enumerate(img_tags, start=1):
    img_url = img.get("src")
    if not img_url:
        print(f"警告: {i}番目のimgタグにはsrc属性がありませんでした。スキップします。")
        continue  # src属性がない場合はスキップ

    # 相対URLを絶対URLに変換
    img_url_absolute = urljoin(TARGET_URL, img_url)

    # 画像データ取得と保存
    try:
        print(f"({i}/{len(img_tags)}) ダウンロード中: {img_url_absolute}")
        img_data = requests.get(img_url_absolute, timeout=10).content
        
        # ファイル名を決定（元のファイル名を取得しようと試みる）
        filename_from_url = os.path.basename(img_url_absolute)
        # クエリパラメータなどを削除
        filename_from_url = filename_from_url.split('?')[0]
        
        if filename_from_url:
            filename = f"{i:03d}_{filename_from_url}"
        else:
            # URLからファイル名が取得できない場合は連番を振る
            filename = f"image_{i:03d}.jpg" # デフォルト拡張子

        save_path = os.path.join(SAVE_DIR, filename)

        # 書き込み
        with open(save_path, "wb") as f:
            f.write(img_data)
        # print(f"保存成功: {save_path}") # 詳細すぎるのでコメントアウト
        saved_count += 1
        
        # サーバーに負荷をかけすぎないための待機時間
        time.sleep(0.5)

    except requests.exceptions.RequestException as e:
        print(f"エラー: {img_url_absolute} のダウンロードに失敗しました: {e}")
    except Exception as e:
        print(f"エラー: {img_url_absolute} の保存中に予期せぬエラーが発生しました: {e}")

print(f"\n完了しました。{saved_count}個の画像を'{SAVE_DIR}'に保存しました。")